<a href="https://colab.research.google.com/github/divsal009/div/blob/master/HOMEAI_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [2]:
!pip install fuzzywuzzy


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from flask import Flask, request, render_template_string
import re
from transformers import pipeline

# Initialize the Flask app
app = Flask(__name__)

# Load the Hugging Face GPT-2 model
generator = pipeline('text-generation', model='gpt2')

# Preprocessing helper function
def preprocess_text(text):
    """Normalize the text: remove special characters, extra spaces, and convert to lowercase."""
    if text is None:
        return ""
    return re.sub(r'[^a-zA-Z0-9\s]', '', text.strip().lower())

# Helper function to find the closest match with fuzzy matching
def find_closest_match(input_text, text_list, threshold=80):
    """Find the closest match for the input text within the text list, using fuzzy matching."""
    if input_text in text_list:
        return input_text
    match = process.extractOne(input_text, text_list, scorer=fuzz.partial_ratio)
    return match[0] if match and match[1] >= threshold else None

# HTML Template for the Home Page
home_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Flask with GPT-2</title>
    <style>
        body { font-family: 'Arial', sans-serif; background-color: #d4f7d4; color: #333; margin: 0; padding: 0; }
        .navbar { background: #0056b3; padding: 15px 0; display: flex; justify-content: center; align-items: center; text-align: center; }
        .navbar a { color: white; text-decoration: none; font-weight: bold; margin: 0 15px; font-size: 18px; }
        .navbar a:hover { text-decoration: underline; color: #ffdd57; }
        .container { max-width: 800px; margin: 50px auto; text-align: center; color: #333; }
        h1 { color: #0056b3; }
        p { font-size: 20px; }
        input[type="text"] { width: 100%; padding: 15px; font-size: 22px; margin: 15px 0; border: 2px solid #ddd; border-radius: 5px; }
        button { padding: 15px 30px; font-size: 22px; background-color: #0056b3; color: white; border: none; border-radius: 5px; cursor: pointer; }
        button:hover { background-color: #003d75; }
    </style>
</head>
<body>
    <div class="navbar">
        <a href="/">Home</a>
        <a href="/disease">Enter Disease Details</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="container">
        <h1>Welcome to the GPT-2 Flask App</h1>
        <h2>Enter a text prompt to generate more content!</h2>
        <form method="POST">
            <input type="text" name="input_text" placeholder="Enter a prompt here..." required>
            <button type="submit">Generate Text</button>
        </form>
        {% if generated_text %}
            <h3>Generated Text:</h3>
            <p>{{ generated_text }}</p>
        {% endif %}
    </div>
</body>
</html>
"""

# HTML Template for the Disease Details Page (Optional for your use case)
disease_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Disease Details - Flask App</title>
</head>
<body>
   <h1>Enter Disease or Symptom</h1>
   <form method="POST">
       <input type="text" name="disease_name" placeholder="Enter disease or symptom" required>
       <button type="submit">Search</button>
   </form>
   {% if message %}
       <p>{{ message }}</p>
   {% endif %}
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def home():
    generated_text = None
    if request.method == 'POST':
        input_text = request.form['input_text']
        # Generating additional text based on the prompt using GPT-2
        generated_text = generator(input_text, max_length=150, num_return_sequences=1)[0]['generated_text']
    return render_template_string(home_template, generated_text=generated_text)

@app.route('/disease', methods=['GET', 'POST'])
def disease():
    message = None
    if request.method == 'POST':
        disease_name = request.form['disease_name']
        message = f"You searched for disease: {disease_name}."
    return render_template_string(disease_template, message=message)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, port=5019)


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5019
INFO:werkzeug:Press CTRL+C to quit
